[View in Colaboratory](https://colab.research.google.com/gist/naviarh/fda21c0840a564fe39a7d26688f448b9/fastai_rnn2.ipynb)

In [0]:
!pip install --upgrade pip
!pip3 install fastai

### Настройка платформы

In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!pwd

/content


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [4]:
import subprocess, os
os.uname()

posix.uname_result(sysname='Linux', nodename='2d49e9e490a7', release='4.14.33+', version='#1 SMP Wed Jun 20 01:36:48 PDT 2018', machine='x86_64')

In [5]:
from fastai.io import *
from fastai.structured import *
from fastai.column_data import *
from fastai.conv_learner import *
!pip3 show fastai torch | grep Name -A 1

Name: fastai
Version: 0.7.0
--
Name: torch
Version: 0.3.1


In [6]:
torch.cuda.set_device(0)
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
#from fastai.imports import *
#from fastai.transforms import *
#from fastai.conv_learner import *
#from fastai.model import *
#from fastai.dataset import *
#from fastai.sgdr import *
#from fastai.plots import *
#!pip3 show fastai torch | grep Name -A 1

### Начало

Мы собираемся скачать собрание сочинений Ницше в качестве данных для этого класса.

In [0]:
PATH='nietzsche/'
os.makedirs(PATH, exist_ok=True)

In [8]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

nietzsche.txt: 606kB [00:00, 1.66MB/s]                           

corpus length: 600893


In [9]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [10]:
chars = sorted(list(set(text))) # отсортированный список множества всех символов
vocab_size = len(chars)+1 # количество символов (+1 для нулевого значения)
print('total chars:', vocab_size)

total chars: 85


In [11]:
len(set(text))

84

In [12]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [13]:
# Вставим нулевое значение в набор для отступов
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [14]:
print(chars)

['\x00', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [15]:
len(chars)

85

In [0]:
# Карта символов в индексы
char_indices = {c: i for i, c in enumerate(chars)}
# Карта индексов в символы
indices_char = {i: c for i, c in enumerate(chars)}

In [17]:
print(char_indices)

{'\x00': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '=': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, 'Æ': 80, 'ä': 81, 'æ': 82, 'é': 83, 'ë': 84}


In [18]:
print(indices_char)

{0: '\x00', 1: '\n', 2: ' ', 3: '!', 4: '"', 5: "'", 6: '(', 7: ')', 8: ',', 9: '-', 10: '.', 11: '0', 12: '1', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: ':', 22: ';', 23: '=', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28: 'D', 29: 'E', 30: 'F', 31: 'G', 32: 'H', 33: 'I', 34: 'J', 35: 'K', 36: 'L', 37: 'M', 38: 'N', 39: 'O', 40: 'P', 41: 'Q', 42: 'R', 43: 'S', 44: 'T', 45: 'U', 46: 'V', 47: 'W', 48: 'X', 49: 'Y', 50: 'Z', 51: '[', 52: ']', 53: '_', 54: 'a', 55: 'b', 56: 'c', 57: 'd', 58: 'e', 59: 'f', 60: 'g', 61: 'h', 62: 'i', 63: 'j', 64: 'k', 65: 'l', 66: 'm', 67: 'n', 68: 'o', 69: 'p', 70: 'q', 71: 'r', 72: 's', 73: 't', 74: 'u', 75: 'v', 76: 'w', 77: 'x', 78: 'y', 79: 'z', 80: 'Æ', 81: 'ä', 82: 'æ', 83: 'é', 84: 'ë'}


In [19]:
# Данные текста (text) в виде индексов
idx = [char_indices[c] for c in text]
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [20]:
# Пример получения строки стекста по карте индексов
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

Теперь наш датасет - переменная **idx**

### Рекуррентная модель

In [0]:
# Длина рекурсии
cs=8

In [0]:
# Создадим набор из 8 последовательностей каждой 8 буквы со смещением
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]

In [0]:
# Выходная последовательность подобным же образом
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [24]:
# наши входы
xs = np.stack(c_in_dat, axis=0)
xs.shape

(600885, 8)

In [25]:
# наш выход
y = np.stack(c_out_dat)
y.shape

(600885,)

In [26]:
xs[:cs,:cs]

array([[40, 42, 29, 30, 25, 27, 29,  1],
       [42, 29, 30, 25, 27, 29,  1,  1],
       [29, 30, 25, 27, 29,  1,  1,  1],
       [30, 25, 27, 29,  1,  1,  1, 43],
       [25, 27, 29,  1,  1,  1, 43, 45],
       [27, 29,  1,  1,  1, 43, 45, 40],
       [29,  1,  1,  1, 43, 45, 40, 40],
       [ 1,  1,  1, 43, 45, 40, 40, 39]])

In [27]:
y[:cs]

array([ 1,  1, 43, 45, 40, 40, 39, 43])

### Создание и обучение модели

In [0]:
# Валидационный набор
val_idx = get_cv_idxs(len(idx)-cs-1)

In [0]:
# Модель данных
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [0]:
# Размер эмбеддинга
n_fac = 42

In [0]:
# Размер скрытого слоя
n_hidden = 256

In [0]:
# Описание модели

class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac) # матрица эмбеддинга
        self.l_in = nn.Linear(n_fac, n_hidden) # входная матрица
        self.l_hidden = nn.Linear(n_hidden, n_hidden) # матрица скрытого слоя
        self.l_out = nn.Linear(n_hidden, vocab_size) # выходная матрица
        
    def forward(self, *cs):
        bs = cs[0].size(0) # 
        h = V(torch.zeros(bs, n_hidden).cuda()) # 
        for c in cs:
            inp = F.relu(self.l_in(self.e(c))) # входная последовательность функций
            h = F.tanh(self.l_hidden(h+inp)) # функция срытого слоя
        
        return F.log_softmax(self.l_out(h), dim=-1) # функция выходного слоя

In [33]:
# Модель
m = CharLoopModel(vocab_size, n_fac).cuda()
m

CharLoopModel(
  (e): Embedding(85, 42)
  (l_in): Linear(in_features=42, out_features=256, bias=True)
  (l_hidden): Linear(in_features=256, out_features=256, bias=True)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [0]:
# Метод оптимизации
opt = optim.Adam(m.parameters(), 1e-2)

In [35]:
# Оптимизацция
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.031783   2.046997  



[array([2.047])]

In [0]:
# Изменяем скорость обучения
set_lrs(opt, 0.001)

In [37]:
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.766367   1.768508  



[array([1.76851])]

### Конкатенационная модель

In [0]:
# Здесь объединяются данные (конкатенация) вместо сложения
# Обратная связь уже от выхода скрытого слоя ко входу входного
class CharLoopConcatModel(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
        
        return F.log_softmax(self.l_out(h), dim=-1)

In [39]:
# Модель и оптимизацитор
m = CharLoopConcatModel(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)
m

CharLoopConcatModel(
  (e): Embedding(85, 42)
  (l_in): Linear(in_features=298, out_features=256, bias=True)
  (l_hidden): Linear(in_features=256, out_features=256, bias=True)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [0]:
#it = iter(md.trn_dl)
#*xs,yt = next(it)
#t = m(*V(xs))

In [40]:
# Оптимизация
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.815643   1.797249  



[array([1.79725])]

In [0]:
set_lrs(opt, 1e-4)

In [42]:
fit(m, md, 3, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.721299   1.716826  
 22%|██▏       | 204/939 [00:07<00:25, 28.75it/s, loss=1.7]

    1      1.681616   1.685233  
    2      1.657473   1.659144  



[array([1.65914])]

### PyTorch модель

In [0]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        # PyTorch не перезаписывает переменную "h",
        # а дописывает новый слой в тензор скрытых слоёв "outp"
        # outp[-1] - это последний слой тензора
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [44]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)
m

CharRnn(
  (e): Embedding(85, 42)
  (rnn): RNN(42, 256)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [0]:
#it = iter(md.trn_dl)
#*xs,yt = next(it)

In [0]:
#t = m.e(V(torch.stack(xs)))
#t.size()

In [0]:
#ht = V(torch.zeros(1, 512,n_hidden))
#outp, hn = m.rnn(t, ht)
#outp.size(), hn.size()

In [0]:
#t = m(*V(xs))
#t.size()

In [45]:
fit(m, md, 4, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.887871   1.865992  
 22%|██▏       | 211/939 [00:06<00:21, 34.61it/s, loss=1.82]

    1      1.692757   1.683696  


    2      1.601237   1.602984  
    3      1.536293   1.557753  



[array([1.55775])]

In [0]:
set_lrs(opt, 1e-4)

In [47]:
fit(m, md, 5, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.468604   1.516624  
 22%|██▏       | 211/939 [00:05<00:20, 35.34it/s, loss=1.47]

    1      1.461387   1.511086  


    2      1.453426   1.50749   


    3      1.447574   1.503609  
    4      1.448536   1.501392  



[array([1.50139])]

### Результаты

In [0]:
# Функция прогнозирования
def get_next(inp):
  # Три символа заменяются на индексы
  # и оборачиваются в массив, после в тензор
    idxs = T(np.array([char_indices[c] for c in inp]))
  # затем в Variable и передаются модели
    p = m(*VV(idxs))
  # Возвращаем самое вероятное предсказание
  # в формате numpy
    i = np.argmax(to_np(p))
    return chars[i]

In [54]:
get_next('pea')

's'

In [59]:
get_next('sig')

'n'

In [60]:
get_next('. T')

'h'

In [0]:
# Создадим генератор текста
def gen_text(inp, n=1):
  result = inp
  for _ in range(n):
    simb = get_next(inp)
    inp = inp[1:] + simb
    result += simb
  return result

In [62]:
gen_text('The', n=50)

'Theress of and of and of and of and of and of and of '

In [0]:
# Создадим генератор слов
def gen_word(inp):
  result = inp
  for _ in range(10):
    simb = get_next(inp)
    if simb == ' ': break
    inp = inp[1:] + simb
    result += simb
  return result

In [63]:
gen_word('app')

'appess'

In [64]:
gen_word('wor')

'word,'

In [68]:
gen_word('pro')

'problems'

### Модель с мультивыходом

In [0]:
# Входной набор 8 последовательностей
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs-1)]
# Выходной набор 8 последовательностей
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs)]

In [70]:
xs = np.stack(c_in_dat)
xs.shape

(600884, 8)

In [71]:
ys = np.stack(c_out_dat)
ys.shape

(600884, 8)

In [0]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [0]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [0]:
class CharLoopConcatModelMultiout(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        res = []
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
            res.append(F.log_softmax(self.l_out(h), dim=-1))
        
        return torch.stack(res)

In [75]:
m = CharLoopConcatModelMultiout(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)
m

CharLoopConcatModelMultiout(
  (e): Embedding(85, 42)
  (l_in): Linear(in_features=298, out_features=256, bias=True)
  (l_hidden): Linear(in_features=256, out_features=256, bias=True)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [0]:
# Своя функция потерь.
# Приниимает на входе 2 минибатча векторов (тензоров второго ранга),
# а у нас прогноз 8 символов, для каждого 84% вероятностей символов,
# и так 512 раз в наборе минибатча (это тензор третьего ранга).
# Функция перекрёстной энтропии от PyTorch будет выдавать ошибку
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size() # размеры тензора предсказаний
    # sl - глубина рекурсии (8)
    # bs - размер минибатча (512)
    # nh - размер словаря (85)
    # размер inp [4, 512, 85]
    # размер targ [512, 4]
    # транспортируем target и перезаписываем (contiguous())
    targ = targ.transpose(0,1).contiguous() # теперь [4, 512]
    # Теперь возвращаем функцию перекрёстной энтропии
    # аргументы которой - копии преобразованных тензоров(view())
    # -1 - это отсутствие ограничений длины результата
    return F.nll_loss(inp.view(-1,nh), targ.view(-1)) # nh - ограничеваем по этой оси

In [77]:
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.910652   1.89644   
 21%|██        | 197/939 [00:07<00:26, 28.11it/s, loss=1.87]

    1      1.790027   1.785903  


    2      1.731856   1.727102  


    3      1.696875   1.699761  
    4      1.676653   1.678817  



[array([1.67882])]

### Теперь разбиваем данные без наложений

In [0]:
cs=4

In [0]:
# Наборы входных и выходных данных без наложения
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(0, len(idx)-cs-1, cs)]
c_out_dat = [[idx[i+j] for i in range(cs)] for j in range(1, len(idx)-cs, cs)]

In [86]:
xs = np.stack(c_in_dat)
xs.shape

(150222, 4)

In [87]:
ys = np.stack(c_out_dat)
ys.shape

(150222, 4)

In [88]:
xs[:cs,:cs]

array([[40, 42, 29, 30],
       [25, 27, 29,  1],
       [ 1,  1, 43, 45],
       [40, 40, 39, 43]])

In [89]:
ys[:cs,:cs]

array([[42, 29, 30, 25],
       [27, 29,  1,  1],
       [ 1, 43, 45, 40],
       [40, 39, 43, 33]])

In [0]:
val_idx = get_cv_idxs(len(xs)-cs-1)

In [0]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, ys, bs=512)

In [0]:
class CharLoopConcatModelMultiout2(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac+n_hidden, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        res = []
        for c in cs:
            inp = torch.cat((h, self.e(c)), 1)
            inp = F.relu(self.l_in(inp))
            h = F.tanh(self.l_hidden(inp))
            res.append(F.log_softmax(self.l_out(h), dim=-1))
        
        return torch.stack(res)

In [95]:
m = CharLoopConcatModelMultiout2(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)
m

CharLoopConcatModelMultiout2(
  (e): Embedding(85, 42)
  (l_in): Linear(in_features=298, out_features=256, bias=True)
  (l_hidden): Linear(in_features=256, out_features=256, bias=True)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [0]:
#it = iter(md.trn_dl)
#*xst,yt = next(it)

In [0]:
# Своя функция потерь
# 6 видеолекция (2:02:00)
def nll_loss_seq(inp, targ):
    sl,bs,nh = inp.size() # размеры тензора предсказаний
    targ = targ.transpose(0,1).contiguous().view(-1)
    return F.nll_loss(inp.view(-1,nh), targ)

In [98]:
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.367091   2.290382  
    1      2.176948   2.155048  
    2      2.098328   2.096423  
  6%|▌         | 13/235 [00:00<00:06, 32.46it/s, loss=2.09]

    3      2.056387   2.060167  
    4      2.030053   2.042682  



[array([2.04268])]

In [0]:
set_lrs(opt, 1e-2)

In [100]:
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.166939   2.135962  
    1      2.099563   2.101177  
    2      2.073298   2.08638   
  6%|▌         | 13/235 [00:00<00:06, 31.99it/s, loss=2.07]

    3      2.069094   2.079043  
    4      2.060921   2.082815  



[array([2.08281])]

In [0]:
set_lrs(opt, 1e-3)

In [102]:
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.959623   1.985201  
    1      1.949432   1.977318  
    2      1.944148   1.970504  
  6%|▌         | 13/235 [00:00<00:06, 32.37it/s, loss=1.94]

    3      1.928895   1.967703  
    4      1.92444    1.966207  



[array([1.96621])]

In [103]:
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.924482   1.962492  
    1      1.915821   1.958843  
    2      1.912758   1.957122  
  6%|▌         | 13/235 [00:00<00:06, 32.55it/s, loss=1.91]

    3      1.913359   1.954231  
    4      1.906833   1.95323   



[array([1.95323])]

### PyTorch модель с мультивыходом

In [0]:
class CharSeqRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        return F.log_softmax(self.l_out(outp), dim=-1)

In [111]:
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-2)
m

CharSeqRnn(
  (e): Embedding(85, 42)
  (rnn): RNN(42, 256)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [112]:
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.182635   2.165476  
    1      2.109993   2.128109  
    2      2.07522    2.09119   
  7%|▋         | 16/235 [00:00<00:05, 38.84it/s, loss=2.07]

    3      2.062829   2.079025  
    4      2.050527   2.067003  



[array([2.067])]

In [0]:
set_lrs(opt, 1e-3)

In [114]:
fit(m, md, 10, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.936969   1.987199  
    1      1.927007   1.978889  
    2      1.923454   1.974437  
  9%|▉         | 21/235 [00:00<00:05, 42.50it/s, loss=1.92]

    3      1.913069   1.973183  
    4      1.912672   1.969758  
    5      1.905313   1.968144  
 56%|█████▌    | 132/235 [00:02<00:02, 46.44it/s, loss=1.9]

    6      1.906057   1.968326  
    7      1.896731   1.967235  
    8      1.894914   1.964288  
 73%|███████▎  | 172/235 [00:03<00:01, 45.78it/s, loss=1.89]

    9      1.889866   1.964123  



[array([1.96412])]

### Результаты

In [0]:
# Функция прогнозирования
def getnext(inp):
  # Три символа заменяются на индексы
  # и оборачиваются в массив, после в тензор
    idxs = T(np.array([char_indices[c] for c in inp]))
  # затем в Variable и передаются модели
    p = m(*VV(idxs))
  # Возвращаем самое вероятное предсказание
  # в формате numpy
    i = np.argmax(to_np(p))
    out = ''
    for i in range(len(p)):
      out += chars[np.argmax(to_np(p[i]))]
    return out

**Пробуем**

In [116]:
getnext('for')

' r '

In [117]:
getnext('par')

'ert'

In [118]:
getnext('que')

'ues'

In [0]:
# Создадим генератор текста
def gentext(inp, cs, n=1):
  result = inp[:cs]
  for _ in range(n):
    simb = getnext(inp)
    inp = inp[1:] + simb
    result += simb
  return result

In [120]:
gentext('On ', cs, n=5)

'On net toe toc th to shoh toh roheth  hth toh roheth taoueah ateoerhea'

### Предотвращение градиентного взрыва

In [145]:
# При создании модели матрицы наполняются рандомом
m = CharSeqRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)
m

CharSeqRnn(
  (e): Embedding(85, 42)
  (rnn): RNN(42, 256)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [146]:
# Инициализация скрытого слоя self.l_hidden (nn.Linear(n_hidden, n_hidden))
# не рандомом а единичной разряженной матрицей - улучшает модель
# Адрес поля матрицы "weight_hh_l0" можно посмотреть в справке "?m.rnn"
m.rnn.weight_hh_l0.data.copy_(torch.eye(n_hidden))


    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.cuda.FloatTensor of size 256x256 (GPU 0)]

In [0]:
# Теперь скорость обучения можно увеличить
set_lrs(opt, 1e-2)

In [148]:
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      2.195997   2.162389  
    1      2.111458   2.115901  
    2      2.08745    2.106427  
  7%|▋         | 16/235 [00:00<00:05, 39.42it/s, loss=2.08]

    3      2.061268   2.092948  
    4      2.057216   2.08431   



[array([2.08431])]

In [150]:
set_lrs(opt, 1e-3)
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.942178   1.992948  
    1      1.928569   1.983297  
    2      1.917704   1.979764  
  9%|▉         | 21/235 [00:00<00:04, 44.74it/s, loss=1.91]

    3      1.915166   1.976594  
    4      1.911268   1.974376  



[array([1.97438])]

In [151]:
fit(m, md, 5, opt, nll_loss_seq)
set_lrs(opt, 1e-4)
fit(m, md, 5, opt, nll_loss_seq)

epoch      trn_loss   val_loss   
    0      1.90737    1.973441  
    1      1.90494    1.972056  
    2      1.901813   1.970482  
  9%|▉         | 21/235 [00:00<00:05, 38.21it/s, loss=1.9]

    3      1.898568   1.969391  
    4      1.892412   1.969066  



epoch      trn_loss   val_loss   
    0      1.873604   1.961711  
    1      1.876677   1.961037  
    2      1.877795   1.960856  
  7%|▋         | 17/235 [00:00<00:05, 42.21it/s, loss=1.87]

    3      1.871393   1.960652  
    4      1.874141   1.960447  



[array([1.96045])]

**Пробуем**

In [152]:
getnext('for')

' r '

In [154]:
getnext('you')

' ur'

In [155]:
gentext('On ', cs, n=5)

'On Net taretor thsi Tvhthst hryohe ot hhhkt hrythe otreeee oaae ne wfh'